In [1]:
from search import *
from game import *
from util import *
from pacman import *
import util, layout
import sys, types, time, random, os
import numpy as np
from random import random, randint, seed, choice


In [2]:
# tiny genetic programming by © moshe sipper, www.moshesipper.com

from statistics import mean
from copy import deepcopy

POP_SIZE        = 60   # population size
MIN_DEPTH       = 2    # minimal initial random tree depth
MAX_DEPTH       = 5    # maximal initial random tree depth
GENERATIONS     = 250  # maximal number of generations to run evolution
TOURNAMENT_SIZE = 5    # size of tournament for tournament selection
XO_RATE         = 0.8  # crossover rate 
PROB_MUTATION   = 0.2  # per-node mutation probability 



def add(x, y): return x + y
def sub(x, y): return x - y
def mul(x, y): return x * y

"FUNCOES ( sao as operacoes executadas nos nós das arvores)-> add, sub, mul, div (x/0 = 1)*"
"TERMINAIS ( sao os tipos possiveis de nohs folha )"
'''
    TERMINAIS ( sao os tipos possiveis de nohs folha )
        -> GAMESTATE_X, onde 0 <= X < Num. de variaveis de estado do jogo
        -> CONSTANTS ( Uns numeros constantes para algumas operações, para facilitar o calculo numerico (?) )
        
    
'''




FUNCTIONS = [add, sub, mul]
TERMINALS = ['actionScore', 'eatFood', 'minGhostDist', 'minDistToFood', 'distToPill', -1.0, 0.5, 1.0, 2.0] # 0.0,  -0.5,

STATE_VARS = ['actionScore', 'eatFood', 'minGhostDist', 'minDistToFood', 'distToPill']
" actionScore: scoreFuturo - scoreAtual "
" eatFood: numFoodAtual - numFoodFuturo"
" minGhostDist: |ghostX - pacmanX| + |ghostY - pacmanY| "
" minDistToFood: |foodX - pacmanX| + |foodY - pacmanY| "
" distToPill: |pillX - pacmanX| + |pillY - pacmanY| "


class GPTree:
    def __init__(self, data = None, left = None, right = None):
        self.data  = data
        self.left  = left
        self.right = right
        
    def setLeft(self, tree):
        self.left = tree
        
    def setRight(self, tree):
        self.right = tree
        
    def node_label(self): # string label
        if (self.data in FUNCTIONS):
            return self.data.__name__
        else: 
            return str(self.data)
    
    def print_tree(self, prefix = ""): # textual printout
        print("%s%s" % (prefix, self.node_label()))        
        if self.left:  self.left.print_tree (prefix + "   ")
        if self.right: self.right.print_tree(prefix + "   ")

    def computeTree(self, stateVars): 
        if (self.data in FUNCTIONS): 
            return self.data(self.left.computeTree(stateVars), self.right.computeTree(stateVars))
        elif (self.data in STATE_VARS): 
            return stateVars[self.data]
        else: return self.data
        
    def size(self): # tree size in nodes
        if self.data in TERMINALS: return 1
        l = self.left.size()  if self.left  else 0
        r = self.right.size() if self.right else 0
        return 1 + l + r
    
    def random_tree(self, grow, max_depth, depth = 0): # create random tree using either grow or full method
        if depth < MIN_DEPTH or (depth < max_depth and not grow): 
            self.data = FUNCTIONS[randint(0, len(FUNCTIONS)-1)]
        elif depth >= max_depth:   
            self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
        else: # intermediate depth, grow
            if random () > 0.5: 
                self.data = TERMINALS[randint(0, len(TERMINALS)-1)]
            else:
                self.data = FUNCTIONS[randint(0, len(FUNCTIONS)-1)]
        if self.data in FUNCTIONS:
            self.left = GPTree()          
            self.left.random_tree(grow, max_depth, depth = depth + 1)            
            self.right = GPTree()
            self.right.random_tree(grow, max_depth, depth = depth + 1)
            
    def getSubtree(self):
        t = GPTree()
        t.data = self.data
        if self.left:  t.left  = self.left.getSubtree()
        if self.right: t.right = self.right.getSubtree()
        return t
    
    def scan_tree(self, count, second): # note: count is list, so it's passed "by reference"
        count[0] -= 1            
        if count[0] <= 1: 
            if not second: # return subtree rooted here
                return self.getSubtree()
            else: # glue subtree here
                self.data  = second.data
                self.left  = second.left
                self.right = second.right
        else:  
            ret = None              
            if self.left  and count[0] > 1: ret = self.left.scan_tree(count, second)  
            if self.right and count[0] > 1: ret = self.right.scan_tree(count, second)  
            return ret
        
    def mutation(self, probability=0.1):
        if random() < probability: # mutate at this node
            self.random_tree(grow = True, max_depth = 2)
        elif self.left: self.left.mutation(probability)
        elif self.right: self.right.mutation(probability)
        

In [3]:
tree = GPTree()
tree.random_tree(True, 2)
tree.print_tree()
thisdict = {
    STATE_VARS[0]: 1.0,
    STATE_VARS[1]: 2.0,
    STATE_VARS[2]: 3.0,
    STATE_VARS[3]: 4.0,
    STATE_VARS[4]: 1.0,
}
print(tree.computeTree(thisdict))

mul
   sub
      distToPill
      minGhostDist
   sub
      distToPill
      eatFood
2.0


In [4]:
" Criar populacao inicial"
''' Para a criação da população inicial, utilizamos o método RandomTree() que gera árvores randomicamente dentro 
    do domínio de operações fundamentais (add, sub, mul) 
        e dos terminais definidos (variaveis de estado e constantes)
'''

' Para a cria\xc3\xa7\xc3\xa3o da popula\xc3\xa7\xc3\xa3o inicial, utilizamos o m\xc3\xa9todo RandomTree() que gera \xc3\xa1rvores randomicamente dentro \n    do dom\xc3\xadnio de opera\xc3\xa7\xc3\xb5es fundamentais (add, sub, mul) \n        e dos terminais definidos (variaveis de estado e constantes)\n'

In [5]:

def initialPopulation(populationSize=32,treeDepth=3):
    population = []
    for _ in range(populationSize):
        tree = GPTree()
        tree.random_tree(True, treeDepth)
        population.append(tree)
    return population

In [6]:
" FITNESS "

# stateVars -> dictionary  ex stateVars['distX'] = distXGameStateVariable
def getFitness(individuo):
    return trainIndividual(individuo)

def getPopFitness(population):
    return [getFitness(population[i]) for i in range(0,len(population))]

    
    

" SELEÇÂO "
'''Para algotimo de seleção, adotaremos inicialmente o método de seleção por Torneio, na ideia de
    aumentar as randomicidade do modelo.
'''
def tournament(population, fitness, size):
    indexes = np.random.choice(range(0,len(population)),size,False)
    chosen = [fitness[i] for i in indexes]
    return population[indexes[chosen.index(max(chosen))]]

def getParent(population, fitness,size=4):
    return tournament(population, fitness, size)


" CROSSOVER "
''' Para a reprodução dos pais, utilizamos uma função de crossover para árvores, 
'''

def crossover(treeA, treeB): # xo 2 trees at random nodes
#     if random() < XO_RATE:
#         geneA = treeA.scan_tree([randint(1, treeA.size())], None) # 1st random subtree
#         geneB = treeB.scan_tree([randint(1, treeB.size())], None) # 2nd random subtree
#         treeA.scan_tree([randint(1, treeA.size())], geneB) # 2nd subtree "glued" inside 1st tree
#         treeB.scan_tree([randint(1, treeB.size())], geneA) # 1st subtree "glued" inside 2nd tree
        
    child1 = treeA.getSubtree()
    child2 = treeB.getSubtree()
    
    aux = child1.right.getSubtree()
    aux.print_tree()
    child1.right = child2.left.getSubtree()
    child2.left = aux
    
    return child1, child2
    
            
" Loop de Reproducao"


' Loop de Reproducao'

In [7]:
tree = GPTree()
tree.random_tree(True, 3)
tree.print_tree()

tree2 = GPTree()
tree2.random_tree(True, 3)
tree2.print_tree()

c1, c2 = crossover(tree, tree2)
c1.print_tree()
c2.print_tree()

add
   sub
      mul
         -1.0
         -1.0
      mul
         0.5
         minDistToFood
   sub
      minGhostDist
      mul
         actionScore
         0.5
mul
   add
      minGhostDist
      mul
         distToPill
         eatFood
   mul
      minGhostDist
      sub
         -1.0
         eatFood
sub
   minGhostDist
   mul
      actionScore
      0.5
add
   sub
      mul
         -1.0
         -1.0
      mul
         0.5
         minDistToFood
   add
      minGhostDist
      mul
         distToPill
         eatFood
mul
   sub
      minGhostDist
      mul
         actionScore
         0.5
   mul
      minGhostDist
      sub
         -1.0
         eatFood


In [8]:
n = Directions.NORTH
s = Directions.SOUTH
e = Directions.EAST
w = Directions.WEST

def manhattanDistance(posA, posB):
    return (abs(posB[0] - posA[0]) + abs(posB[1] - posA[1]))

def calcMinGhostDist(pacmanPosition, ghostsPosition):
    dist = 99999
    for ghost in ghostsPosition:
        if dist > manhattanDistance(pacmanPosition, ghost):
            dist = manhattanDistance(pacmanPosition, ghost)
            
    return dist

def manhattanFoodDistance(posA, posX, posY):
    return (abs(posX - posA[0]) + abs(posY - posA[1]))

def calcMinFoodDist(pacmanPosition, foodGrid):
    dist = 99999
    for i in range(1,len(foodGrid[:])-1):
        for j in range(1,len(foodGrid[0][:])-1):
            if foodGrid[i][j] and (dist > manhattanFoodDistance(pacmanPosition, i, j)):
                dist = manhattanFoodDistance(pacmanPosition, i, j)

    return dist

def calcMinPillDist(pacmanPosition, pills):
    return calcMinGhostDist(pacmanPosition, pills)

class MyGoWestAgent(Agent):
    "An agent that goes West until it can't."
    def __init__(self, tree = None):
        self.tree = tree
        
    def setTree(self, tree):
        selt.tree = tree

    def getAction(self, state):
        "The agent receives a GameState (defined in pacman.py)."
#         print(state.getLegalPacmanActions())
#         print(state.getPacmanState())

#         print(state.getPacmanPosition())
#         print(state.getNumFood())
#         print(state.getScore())
        count = 0
        legalActions = state.getLegalPacmanActions()
        legalActions.remove(Directions.STOP) #remove a opcao Stop
        actionScore = []
        stateVars = {}
        for action in legalActions:
            
            futureState = state.generatePacmanSuccessor(action)
            
            futureScore = futureState.getScore() - state.getScore()
            eatFood = state.getNumFood() - futureState.getNumFood()
            ghostDist = calcMinGhostDist(futureState.getPacmanPosition(),futureState.getGhostPositions())
            foodDist = calcMinFoodDist(futureState.getPacmanPosition(),futureState.getFood())
            pillDist = calcMinPillDist(futureState.getPacmanPosition(),futureState.getCapsules())
            stateVars[STATE_VARS[0]] = futureScore
            stateVars[STATE_VARS[1]] = eatFood
            stateVars[STATE_VARS[2]] = ghostDist
            stateVars[STATE_VARS[3]] = foodDist
            stateVars[STATE_VARS[4]] = pillDist

            treeValue = tree.computeTree(stateVars)
            actionScore.append(treeValue)
            
        
        if actionScore.count(max(actionScore)) > 1:
            maxScore = max(actionScore)
            indexes = [i for i, j in enumerate(actionScore) if j == maxScore]
            maxIndex = choice(indexes)
        else:
            maxIndex = actionScore.index(max(actionScore))
            
        
        
        action = legalActions[maxIndex]
        print("Action Score: ",actionScore)
        
#         print("Action: ",action)
#         futureState = state.generatePacmanSuccessor(action)
#         futureScore = futureState.getScore() - state.getScore()
#         eatFood = state.getNumFood() - futureState.getNumFood()
#         ghostDist = calcMinGhostDist(futureState.getPacmanPosition(),futureState.getGhostPositions())
#         foodDist = calcMinFoodDist(futureState.getPacmanPosition(),futureState.getFood())
#         print("STATE VARS: FS: ", futureScore," EF: ", eatFood, " GD: ", ghostDist," FD: ", foodDist, " PD: ", pillDist)
        
        if action in state.getLegalPacmanActions():
            return action
        else:
            return Directions.STOP

In [9]:
def trainIndividual(ind, layoutType='smallClassic', numGames=4):
    gameLayout = layout.getLayout( 'smallClassic' )
    print("------------------------------------------ train --------------------")
    ind.print_tree()

    pacman = MyGoWestAgent(ind)

    ghostType = loadAgent('RandomGhost', True)
    ghosts = [ghostType( i+1 ) for i in range( 4 )]

    import textDisplay
    textDisplay.SLEEP_TIME = 0.1
    gameDisplay = textDisplay.PacmanGraphics()

    games = runGames( gameLayout, pacman, ghosts, gameDisplay, numGames, False)
    scores = [game.state.getScore() for game in games]
    wins = [game.state.isWin() for game in games]
    winRate = wins.count(True)/ float(len(wins))
    avgScore = sum(scores) / float(len(scores))
    return avgScore

In [10]:
def generationLoop():
    population = initialPopulation(16,4)
#     fitness = np.zeros((1,len(population)))
    count = 1
    for p in population:
        p.print_tree()
        print(" - - - - ")
    print("G - ",count,"    ============================================  ")
    
    # train population to get fitness
        
    fitness = getPopFitness(population)
    print("G",count," - FITNESS: ", fitness )
    while count <= 6:
        
        
        children = []
        
        for i in range(0,int(len(population)/2)-1):
            father = getParent(population, fitness,3)
            mother = getParent(population, fitness,3)
            
            # TODO - check if parent == mother
            
            child1, child2 = crossover(father, mother)
            children.append(child1)
            children.append(child2)
            

#       children = mutation(children,int(len(children)*0.75))#mutation
        indexes = np.random.choice(range(0,len(children)),int(len(children)*0.6),False)
        for i in indexes:
            children[i].mutation(0.2)
        
        # elitism
        maxIndex = fitness.index(max(fitness))
        children.append(population[maxIndex])
        
        fitness.remove(max(fitness))
        population.remove(population[maxIndex])
        
        maxIndex = fitness.index(max(fitness))
        children.append(population[maxIndex])
        
        count += 1
        
        # train population to get fitness
        print("G",count," - POPULATION")
        
        population = children[:]
        for p in population:
            p.print_tree()
            print(" - - - - ")
            
        fitness = getPopFitness(population)
        print("FITNESS: ", fitness )
        
    return population, fitness

In [11]:
population, fitness = generationLoop()
for p in population:
    p.print_tree()
    print(" - - - - ")
print(fitness)

('G - ', 1, '    ============================================  ')
------------------------------------------ train --------------------
sub
   sub
      add
         minGhostDist
         mul
            minGhostDist
            eatFood
      mul
         minGhostDist
         distToPill
   sub
      sub
         mul
            2.0
            2.0
         0.5
      1.0
%%%%%%%%%%%%%%%%%%%%
%......%G  G%......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........<.........%
%%%%%%%%%%%%%%%%%%%%
Score: 0

('Action Score: ', [0.0, 1.0])
%%%%%%%%%%%%%%%%%%%%
%......% GG %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: 9

('Action Score: ', [5.0, 1.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%GG%%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........> ........%
%%%%%%%%%%%%%%%%%%%%
Score: 8

('Action Score: ', [0.0, 4.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%G %%...%%.%

('Action Score: ', [1.0, 5.0])
%%%%%%%%%%%%%%%%%%%%
%.....G%    %......%
%.%%...%% G%%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: 1

('Action Score: ', [5.0, 0.0])
%%%%%%%%%%%%%%%%%%%%
%....G.%    %......%
%.%%...%%G %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........> ........%
%%%%%%%%%%%%%%%%%%%%
Score: 0

('Action Score: ', [0.0, 5.0])
%%%%%%%%%%%%%%%%%%%%
%...G..%    %......%
%.%%...%%  %%...%%.%
%.%o.%...G....%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: -1

('Action Score: ', [3.0, 0.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%G..%%  %%...%%.%
%.%o.%..G.....%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........> ........%
%%%%%%%%%%%%%%%%%%%%
Score: -2

('Action Score: ', [-2.0, 5.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%.G.%%  %%...%%.%
%.%o.%.G......%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: -3

('Action Score: ', [5.0, 2.0])
%%%%%%%%%%%%%%%%%%%%
%...

('Action Score: ', [5.0, -2.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%......GG%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........> ........%
%%%%%%%%%%%%%%%%%%%%
Score: -8

('Action Score: ', [2.0, 5.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%.......G%.o%.%
%.%%.%.%%%%%%G%.%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: -9

('Action Score: ', [6.0, -1.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%G%.%%.%
%........> ..G.....%
%%%%%%%%%%%%%%%%%%%%
Score: -10

('Action Score: ', [1.0, 4.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ <.GG.....%
%%%%%%%%%%%%%%%%%%%%
Score: -11

('Action Score: ', [4.0, -3.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........> GG......%
%%%%%%%%%%%%%%%%%%%%
Score: -12

('Action Score: ', [-1.0, 2.0])
%%%%%%%%%%%%%%%%%

('Action Score: ', [2.0, 5.0])
%%%%%%%%%%%%%%%%%%%%
%......%  G %......%
%.%%...%% G%%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: -11

('Action Score: ', [5.0, 0.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%GG%%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........> ........%
%%%%%%%%%%%%%%%%%%%%
Score: -12

('Action Score: ', [0.0, 4.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%G %%...%%.%
%.%o.%...G....%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: -13

('Action Score: ', [3.0, 0.0])
%%%%%%%%%%%%%%%%%%%%
%......% G  %......%
%.%%...%%  %%...%%.%
%.%o.%....G...%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........> ........%
%%%%%%%%%%%%%%%%%%%%
Score: -14

('Action Score: ', [0.0, 3.0])
%%%%%%%%%%%%%%%%%%%%
%......%  G %......%
%.%%...%%  %%...%%.%
%.%o.%.....G..%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: -15

('Action Score: ', [5.0, -2.0])
%%%%%%%%%%%%%%%%%%

('Action Score: ', [0.0, 3.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%% G%%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ <.....G..%
%%%%%%%%%%%%%%%%%%%%
Score: -55

('Action Score: ', [5.0, 0.0])
%%%%%%%%%%%%%%%%%%%%
%......%  G %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........> ......G.%
%%%%%%%%%%%%%%%%%%%%
Score: -56

('Action Score: ', [2.0, 5.0])
%%%%%%%%%%%%%%%%%%%%
%......% G  %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ <.......G%
%%%%%%%%%%%%%%%%%%%%
Score: -57

('Action Score: ', [5.0, 2.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%G %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%G%
%........> ........%
%%%%%%%%%%%%%%%%%%%%
Score: -58

('Action Score: ', [0.0, 5.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%...G....%.o%G%
%.%%.%.%%%%%%.%.%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: -59

('Action Score: ', [3.0, 0.0])
%%%%%%%%%%%%%%%%%%%

...

('Action Score: ', [4.0, -1.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%G..G....%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........> ........%
%%%%%%%%%%%%%%%%%%%%
Score: 2

('Action Score: ', [-1.0, 4.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%..G.....%.o%.%
%.%%.%G%%%%%%.%.%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: 1

('Action Score: ', [4.0, 1.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%.G......%.o%.%
%.%%.%.%%%%%%.%.%%.%
%.....G..> ........%
%%%%%%%%%%%%%%%%%%%%
Score: 0

('Action Score: ', [-2.0, 5.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%G.......%.o%.%
%.%%.%.%%%%%%.%.%%.%
%......G. <........%
%%%%%%%%%%%%%%%%%%%%
Score: -1

('Action Score: ', [3.0, 0.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%G%%%%%%.%.%%.%
%.......G> ........%
%%%%%%%%%%%%%%%%%%%%
Score: -2

('Action Score: ', [246.0, 3.0])
%%%%%%%%%%%%%%%%%%%%
%

('Action Score: ', [1.0, 5.0])
%%%%%%%%%%%%%%%%%%%%
%......% GG %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: 5

('Action Score: ', [5.0, 1.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%GG%%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........> ........%
%%%%%%%%%%%%%%%%%%%%
Score: 4

('Action Score: ', [0.0, 4.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%G %%...%%.%
%.%o.%...G....%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: 3

('Action Score: ', [3.0, 0.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%...GG...%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........> ........%
%%%%%%%%%%%%%%%%%%%%
Score: 2

('Action Score: ', [-1.0, 3.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%..G..G..%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: 1

('Action Score: ', [4.0, -2.0])
%%%%%%%%%%%%%%%%%%%%
%.....

('Action Score: ', [0.0, 4.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%G %%...%%.%
%.%o.%........%.o%.%
%.%%.%G%%%%%%.%.%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: -39

('Action Score: ', [4.0, 1.0])
%%%%%%%%%%%%%%%%%%%%
%......% G  %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%.....G..> ........%
%%%%%%%%%%%%%%%%%%%%
Score: -40

('Action Score: ', [-2.0, 5.0])
%%%%%%%%%%%%%%%%%%%%
%......%  G %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%....G... <........%
%%%%%%%%%%%%%%%%%%%%
Score: -41

('Action Score: ', [5.0, 1.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%% G%%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%...G....> ........%
%%%%%%%%%%%%%%%%%%%%
Score: -42

('Action Score: ', [0.0, 4.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%....G...%.o%.%
%.%%.%.%%%%%%.%.%%.%
%..G..... <........%
%%%%%%%%%%%%%%%%%%%%
Score: -43

('Action Score: ', [4.0, -1.0])
%%%%%%%%%%%%%%%%%

('Action Score: ', [7.0, 0.0])
%%%%%%%%%%%%%%%%%%%%
%......%  G %......%
%.%%...%%  %%.G.%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........> ........%
%%%%%%%%%%%%%%%%%%%%
Score: 0

('Action Score: ', [2.0, 5.0])
%%%%%%%%%%%%%%%%%%%%
%......% G  %.G....%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: -1

('Action Score: ', [5.0, 2.0])
%%%%%%%%%%%%%%%%%%%%
%......%G   %..G...%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........> ........%
%%%%%%%%%%%%%%%%%%%%
Score: -2

('Action Score: ', [0.0, 7.0])
%%%%%%%%%%%%%%%%%%%%
%......% G  %......%
%.%%...%%  %%..G%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: -3

('Action Score: ', [5.0, 2.0])
%%%%%%%%%%%%%%%%%%%%
%......%  G %......%
%.%%...%%  %%...%%.%
%.%o.%........%Go%.%
%.%%.%.%%%%%%.%.%%.%
%........> ........%
%%%%%%%%%%%%%%%%%%%%
Score: -4

('Action Score: ', [2.0, 5.0])
%%%%%%%%%%%%%%%%%%%%
%...

('Action Score: ', [7.0, 4.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%G%.%%%%%%.%.%%.%
%...G....> ........%
%%%%%%%%%%%%%%%%%%%%
Score: -6

('Action Score: ', [0.0, 7.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%..GG.... <........%
%%%%%%%%%%%%%%%%%%%%
Score: -7

('Action Score: ', [6.0, 3.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%.G..G...> ........%
%%%%%%%%%%%%%%%%%%%%
Score: -8

('Action Score: ', [-1.0, 6.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%G....G.. <........%
%%%%%%%%%%%%%%%%%%%%
Score: -9

('Action Score: ', [4.0, 1.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%G%%.%G%%%%%%.%.%%.%
%........> ........%
%%%%%%%%%%%%%%%%%%%%
Score: -10

('Action Score: ', [-1.0, 6.0])
%%%%%%%%%%%%%%%%%%%%


In [12]:
# args = readCommand(['--layout', 'smallClassic', '--pacman', 'GoWestAgent', '-t']) # Get game components based on input
gameLayout = layout.getLayout( 'smallClassic' )

ind = GPTree()
ind.random_tree(True, 2)
ind.print_tree()

pacman = MyGoWestAgent(ind)
# pacman.setTree(ind)

ghostType = loadAgent('RandomGhost', True)
ghosts = [ghostType( i+1 ) for i in range( 4 )]

import textDisplay
textDisplay.SLEEP_TIME = 0.1
gameDisplay = textDisplay.PacmanGraphics()

runGames( gameLayout, pacman, ghosts, gameDisplay, 4, False)

mul
   sub
      actionScore
      eatFood
   sub
      minDistToFood
      minDistToFood
%%%%%%%%%%%%%%%%%%%%
%......%G  G%......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........<.........%
%%%%%%%%%%%%%%%%%%%%
Score: 0

('Action Score: ', [0.0, 1.0])
%%%%%%%%%%%%%%%%%%%%
%......% GG %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: 9

('Action Score: ', [5.0, 1.0])
%%%%%%%%%%%%%%%%%%%%
%......% GG %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........> ........%
%%%%%%%%%%%%%%%%%%%%
Score: 8

('Action Score: ', [1.0, 5.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%GG%%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: 7

('Action Score: ', [4.0, 0.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%...GG...%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........> ........%
%%%%%%%%%%%%%%%%%%%%
Score: 6

(

('Action Score: ', [-1.0, 2.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ G..G.....%
%%%%%%%%%%%%%%%%%%%%
Score: -519

%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ G..G.....%
%%%%%%%%%%%%%%%%%%%%
Score: -519

Pacman died! Score: -519
%%%%%%%%%%%%%%%%%%%%
%......%G  G%......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........<.........%
%%%%%%%%%%%%%%%%%%%%
Score: 0

('Action Score: ', [0.0, 1.0])
%%%%%%%%%%%%%%%%%%%%
%......% GG %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: 9

('Action Score: ', [5.0, 1.0])
%%%%%%%%%%%%%%%%%%%%
%......%  G %......%
%.%%...%% G%%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........> ........%
%%%%%%%%%%%%%%%%%%%%
Score: 8

('Action Score: ', [1.0, 4.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%GG%%...%

('Action Score: ', [6.0, -1.0])
%%%%%%%%%%%%%%%%%%%%
%...G..%    %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........> ..G.....%
%%%%%%%%%%%%%%%%%%%%
Score: -18

('Action Score: ', [1.0, 4.0])
%%%%%%%%%%%%%%%%%%%%
%....G.%    %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........ <...G....%
%%%%%%%%%%%%%%%%%%%%
Score: -19

('Action Score: ', [6.0, -1.0])
%%%%%%%%%%%%%%%%%%%%
%.....G%    %......%
%.%%...%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%.%%.%
%........> ....G...%
%%%%%%%%%%%%%%%%%%%%
Score: -20

('Action Score: ', [2.0, 6.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%..G%%  %%...%%.%
%.%o.%........%.o%.%
%.%%.%.%%%%%%.%G%%.%
%........ <........%
%%%%%%%%%%%%%%%%%%%%
Score: -21

('Action Score: ', [7.0, 1.0])
%%%%%%%%%%%%%%%%%%%%
%......%    %......%
%.%%...%%  %%...%%.%
%.%o.%G.......%Go%.%
%.%%.%.%%%%%%.%.%%.%
%........> ........%
%%%%%%%%%%%%%%%%%%%%
Score: -22

('Action Score: ', [0.0, 7.0])
%%%%%%%%%%%%%%%%%

[<game.Game instance at 0x7fabd688c0f0>,
 <game.Game instance at 0x7fabd67e7550>,
 <game.Game instance at 0x7fabd6769f50>,
 <game.Game instance at 0x7fabd677ab90>]